In [5]:
import cv2
from ultralytics import YOLO
import tensorflow as tf
import numpy as np
import joblib
from datetime import datetime
import os
import requests
import json
from datetime import datetime, timedelta
import base64
import cvzone
# 初始化模型和其他資源
model = YOLO("yolov8n-pose.pt")
m = "rf"
m_version = '4'
# my_model = tf.keras.models.load_model(f'new_{m}_model_{m_version}.h5')
my_model = joblib.load(f'new_{m}_model_{m_version}.pkl')
my_scaler = joblib.load(f'new_{m}_scaler_{m_version}.pkl')
path = "./fall2.mp4"
cap = cv2.VideoCapture(path)
# LINE Notify token
token = 'ZzU097NT2vxzxwWIELqUuFDFA0WwAQCiwZA4Mc69niR'
notice_time = datetime(1970, 1, 1) 
# 伺服器端點的URL
server_url = 'http://192.168.24.94/api/pictures'
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
frame_count = 0
predict_count = 0
# 將信心值加入關鍵點座標
def main_process(results):
    r = results[0]
    global notice_time
    global predict_count
    global frame_count
    for i in range(len(r.keypoints.xyn)):
        combined_results = []
        for j in range(17):  # Assuming there are 17 keypoints
            x, y = r.keypoints.xyn[i][j].numpy()
            combined_results.append((x, y))
        box = results[0].boxes[i].xywhn.numpy().reshape(-1, 2)
        list_of_tuples = [tuple(row) for row in box]
        combined_results.extend(list_of_tuples)
        data = np.array(combined_results).reshape(1,-1)
        data = my_scaler.transform(data)
        y_pred = my_model.predict(data)
        kp = data.reshape(-1, 2)
        if y_pred[0] == 1:
                predict_count += 1
                if predict_count >= (fps*0.1):
                    x1, y1, x2, y2 = r.boxes[i].xyxy[0].tolist()
                    cvzone.putTextRect(annotated_frame, 'fall', [int(x1), int(y1)], thickness=2, scale=2, colorR=(255, 0, 0))
                    predict_count = 0
                    fall_current_time = current_time
                    folder_name = f'Fall_img_{m}_{m_version}'
                    os.makedirs(folder_name, exist_ok=True)
                    time_text = fall_current_time.strftime("%Y-%m-%dT%H:%M:%S")
                    image_filename = fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
                    cv2.imwrite(os.path.join(folder_name, image_filename), annotated_frame)
                    print("output save")
                    time_difference = notice_time - current_time
                    if notice_time == datetime(1970, 1, 1) or time_difference <= timedelta(seconds=1):
                        image_path = f"./Fall_img_{m}_{m_version}/"+fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
                        send_line_notify(token, image_path=image_path)
                        # send_fall_event_to_server(time_text, kp, image_path)
                        notice_time = current_time + timedelta(seconds=11)
                        print("sent2line")
                        break
                    break
# LINE Notify傳送訊息
def send_line_notify(token, image_path=None):
    message = '有人跌倒了'
    headers = {"Authorization": "Bearer " + token}
    data = {'message': message}
    if image_path:
        with open(image_path, 'rb') as image_file:
            image_data = {'imageFile': image_file}
            response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data, files=image_data)
    else:
        response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
    print(response.status_code)
    print(response.text)
def send_fall_event_to_server(timestamp, kp ,image_path):
    data = {
        'createdAt': timestamp,
        'points': kp.tolist(),
    }
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')    
    data['image'] = encoded_image   
    json_data = json.dumps(data)
    headers = {'Content-Type': 'application/json'}
    response = requests.post(server_url, data=json_data, headers=headers)
    if response.status_code == 200:
        print(f'資料成功發送: {timestamp}')
    else:
        print(f'資料發送失敗: {response.status_code}, {response.text}')
# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model.track(frame, conf=0.7)
    annotated_frame = results[0].plot()
    if len(results[0].boxes) >= 1:
        main_process(results)            
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if frame_count >= fps:
        frame_count = 0
        predict_count = 0
    else:
        frame_count += 1
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 85.6ms
Speed: 1.0ms preprocess, 85.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 87.5ms
Speed: 1.0ms preprocess, 87.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 85.6ms
Speed: 2.0ms preprocess, 85.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 84.5ms
Speed: 1.0ms preprocess, 84.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.5ms
Speed: 1.0ms preprocess, 83.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.4ms
Speed: 1.0ms preprocess, 83.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.6ms
Speed: 1.0ms preprocess, 83.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.9ms
Speed: 1.0ms preprocess, 81.9ms i

In [ ]:
import csv
from ultralytics import YOLO
# Load YOLO model
image_path = "C:/Users/User/Desktop/Fall Detection/detect/detected_9281.jpg"
model = YOLO("yolov8n-pose.pt")
results = model(image_path)
r = results[0] 
# by 照片人數去抓每個人的keypoints 
# 照片人數 = len(results[0].boxes)
for i in range(len(results[0].boxes)):
    # print(r.boxes.xywhn[i])
    print(r.keypoints.conf[i]) # keypoint共有17個點,每個點yolo偵測到會給一個信心值
    print(r.keypoints.xyn[i]) # keypoint共有17個點,yolo偵測到的x,y座標並做標準化
    print("-----------------")

# Initialize a list to store the combined (x, y, confidence) tuples
combined_results = []

# Iterate over each keypoint
for i in range(len(r.keypoints.xyn)):
    for  j  in range(17):
        x, y = r.keypoints.xyn[i][j].numpy() # Convert tensor to numpy
        confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append((x, y, confidence))
print(combined_results)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 209):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/fall{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"fall{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 167):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/not fallen{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"not fallen{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os

directory = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"

missing_files = []

for i in range(1, 209):
    filename = os.path.join(directory, f"fall{i:03d}.jpg")
    if not os.path.exists(filename):
        missing_files.append(i)

if missing_files:
    print("Missing files:")
    print(missing_files)
else:
    print("All files exist.")


In [4]:
    token = '6quBdcwBnxIbx1XjD8GT1InUIoAn0qmsHKxm8MmWPKC'
    # Message to send
    message = '有人跌倒了'

    # HTTP headers and data
    headers = { "Authorization": "Bearer " + token }
    data = { 'message': message }

    # Send the POST request
    response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)

    # Print the response
    print(response.status_code)
    print(response.text)


200
{"status":200,"message":"ok"}


In [20]:
import cv2
from ultralytics import YOLO
import tensorflow as tf
import numpy as np
import joblib
from datetime import datetime
import os
import requests
import json
from datetime import datetime, timedelta
import base64
from collections import deque
import cvzone
# 初始化模型和其他資源
model = YOLO("yolov8n-pose.pt")
m = "rf"
m_version = '3'
# my_model = tf.keras.models.load_model(f'new_{m}_model_{m_version}.h5')
my_model = joblib.load(f'new_{m}_model_{m_version}.pkl')
my_scaler = joblib.load(f'new_{m}_scaler_{m_version}.pkl')
path = "./fall4.mp4"
cap = cv2.VideoCapture(path)
output_video_path = './output_video.mp4'
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
frame_count = 0
predict_count = 0
# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))                    


# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model(frame, conf=0.2)
    frame = results[0].plot()
    r = results[0]
    if len(results[0].boxes) >= 1:
        for i in range(len(r.keypoints.xyn)):
            combined_results = []
            notice_time = datetime(1970, 1, 1)
            for j in range(17):  # Assuming there are 17 keypoints
                x, y = r.keypoints.xyn[i][j].numpy()
                combined_results.append((x, y))
            box = results[0].boxes[i].xywhn.numpy().reshape(-1, 2)
            list_of_tuples = [tuple(row) for row in box]
            combined_results.extend(list_of_tuples)
            data = np.array(combined_results).reshape(1,-1)
            data = my_scaler.transform(data)
            y_pred = my_model.predict(data)
            kp = data.reshape(-1, 2)
            
            if y_pred[0] == 1:
                predict_count += 1
                if predict_count >= (fps*0.8):
                    x1, y1, x2, y2 = r.boxes[i].xyxy[0].tolist()
                    cvzone.putTextRect(frame, 'fall', [int(x1), int(y1)], thickness=2, scale=2, colorR=(255, 0, 0))
                    predict_count = 0
                    break
                break
    # Check predictions every frame
    if frame_count >= fps:
        frame_count = 0
        predict_count = 0
    else:
        frame_count += 1
    out.write(frame)
    print("output saved")
    cv2.imshow('YOLOv8 Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 84.0ms
Speed: 1.0ms preprocess, 84.0ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 83.7ms
Speed: 2.0ms preprocess, 83.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 84.6ms
Speed: 1.0ms preprocess, 84.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 82.3ms
Speed: 2.0ms preprocess, 82.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 82.4ms
Speed: 1.0ms preprocess, 82.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 82.6ms
Speed: 1.0ms preprocess, 82.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
output saved

0: 384x640 (no detections), 81.7ms
Speed: 1.0ms preprocess, 81.7ms inference, 1.0ms postprocess per image at shape (1, 3, 3